
# Код для оптимизации пептидов

Создание окужения Conda для работы всех зависимостей 
markdown что бы код автоматом не выполнялся

!conda create -n bio4 -y
!conda activate bio4
!conda install -c conda-forge mdtraj numpy biopython scikit-learn -y
!conda install -c omnia mdtraj-extras -y

Удалить окружение Conda 

!conda remove --name bio4 --all

## Поиск линейной последовательности

### Импорт необходимых библиотек и функций

In [ ]:
import mdtraj as md
import numpy as np
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio import SeqIO
from Bio.Seq import Seq
from sklearn.preprocessing import MinMaxScaler
from mdtraj.geometry import shrake_rupley

### Загрузка и обработка структуры белка:
__load_structure:__ загружает структуру из PDB-файла  
__identify_cleavage_site:__ ищет место разреза в последовательности белка  
__compute_sasa:__ вычисляет растворимую поверхность атомов (SASA) для каждого остатка белка  
__find_accessible_region:__ определяет доступные для пептидов остатки белка

In [ ]:
def load_structure(file_path):
    return md.load(file_path)

In [ ]:
def identify_cleavage_site(structure, site_sequence):
    protein_seq = structure.top.to_fasta()[0]
    return protein_seq.find(site_sequence)

In [ ]:
def compute_sasa(structure):
    atom_sasa = shrake_rupley(structure, mode='atom')
    atom_sasa = atom_sasa.flatten()
    residue_sasa = np.zeros(structure.n_residues)
    for atom, sasa in zip(structure.top.atoms, atom_sasa):
        residue_sasa[atom.residue.index] += sasa
    return residue_sasa

sasa_threshold = 0.2  # Отрегулируйте по мере необходимости

In [ ]:
def find_accessible_region(structure, cleavage_site, sasa_threshold):
    sasa = compute_sasa(structure)
    accessible_residues = np.where(sasa > sasa_threshold)[0]
    return accessible_residues

In [ ]:
def write_txt_file(filename, peptide):
    with open(filename, "a") as f:
        f.write(peptide + "\n")

### Дизайн кандидатов
__design_candidate_peptides:__ генерирует пептиды кандидаты   
__filter_known_antigenic_motifs:__ фильтрует пептиды, содержащие известные антигенные мотивы

In [ ]:
def design_candidate_peptides(structure, accessible_region, peptide_length_range):
    protein_seq = structure.top.to_fasta()[0]
    candidate_peptides = []
    for start_idx in accessible_region:
        for peptide_length in peptide_length_range:
            end_idx = start_idx + peptide_length
            if end_idx < len(protein_seq):
                candidate_peptides.append(protein_seq[start_idx:end_idx])
    return candidate_peptides

def filter_known_antigenic_motifs(candidate_peptides, antigenic_motifs):
    filtered_peptides = []
    for peptide in candidate_peptides:
        if not any(motif in peptide for motif in antigenic_motifs):
            filtered_peptides.append(peptide)
    return filtered_peptides

known_antigenic_motifs = [
    "AAAA",
    "RRRR",
    # Из литературы реальные записать 
]

### Оценка иммуногенности
__evaluate_immunogenicity:__ оценивает иммуногенност пептидов  
__optimize_peptide_sequence:__ находит топ-N оптимизированных пептидов с наилучшей иммуногенностью

In [ ]:
def evaluate_immunogenicity(peptides):
    # Define the hydrophobicity scale
    hydrophobicity_scale = {
        'A': 0.47, 'R': 0.81, 'N': 0.42, 'D': 1.23, 'C': 0.77, 'Q': 0.58,
        'E': 1.14, 'G': 0.59, 'H': 1.00, 'I': 1.41, 'L': 1.21, 'K': 0.99,
        'M': 1.45, 'F': 1.13, 'P': 0.31, 'S': 0.39, 'T': 0.43, 'W': 1.08,
        'Y': 0.81, 'V': 1.08
    }
    
    scores = []
    for peptide in peptides:
        seq = Seq(peptide)
        pa = ProteinAnalysis(str(seq))
        
        # Calculate instability index
        instability_index = pa.instability_index()

        # Calculate hydrophobicity using the hydrophobicity scale
        hydrophobicity = np.mean([hydrophobicity_scale.get(aa, 0) for aa in peptide])
        
        # Calculate charge
        charge = pa.charge_at_pH(7.4)
        
        # Calculate propensity to form secondary structures
        helix, turn, sheet = pa.secondary_structure_fraction()
        secondary_structure_score = helix + sheet

        # Combine the scores using your preferred weighting
        combined_score = instability_index + hydrophobicity + charge + secondary_structure_score
        scores.append(combined_score)

    scores = np.array(scores)
    scores_min = np.min(scores)
    scores_max = np.max(scores)
    scaled_scores = (scores - scores_min) / (scores_max - scores_min)
    return scaled_scores

In [ ]:
def optimize_peptide_sequence(peptides, immunogenicity_scores, top_n=1):
    top_peptide_indices = np.argsort(immunogenicity_scores)[::-1][:top_n]
    top_peptides = [peptides[i] for i in top_peptide_indices]
    top_scores = [immunogenicity_scores[i] for i in top_peptide_indices]
    return top_peptides, top_scores

### Основной код программы

In [ ]:
if __name__ == "__main__":
    file_path = "c5.pdb"
    site_sequence = "VNND"
    distance_threshold = 10
    peptide_length_range = range(16, 101)  # Generate peptide lengths from 10 to 20 inclusive
    
    structure = load_structure(file_path)
    protein_seq = structure.top.to_fasta()[0]

    print(f"Protein sequence in FASTA format:\n>{structure.topology.chain(0)}\n{protein_seq}")
    
    #cleavage_site = identify_cleavage_site(structure, site_sequence)

    cleavage_site = 73  # Directly set the cleavage site index

    # Set the number of top peptides you'd like to select
    top_n = 5

    if cleavage_site == -1:
        print("Cleavage site sequence not found in the protein. Please ensure the site_sequence variable is correct.")
    else:
        accessible_region = find_accessible_region(structure, cleavage_site, sasa_threshold)
        candidate_peptides = design_candidate_peptides(structure, accessible_region, peptide_length_range)
        candidate_peptides = filter_known_antigenic_motifs(candidate_peptides, known_antigenic_motifs)
        if not candidate_peptides:
            print("No candidate peptides were generated. Please check the input parameters and try again.")
        else:

            try:
                with open("peptides.txt", "w") as file:
                    pass
            except FileNotFoundError:
                pass

            immunogenicity_scores = evaluate_immunogenicity(candidate_peptides)
            optimized_peptides, optimized_scores = optimize_peptide_sequence(candidate_peptides, immunogenicity_scores, top_n)

            print(f"Top {top_n} optimized peptides:")

            for i, (peptide, score) in enumerate(zip(optimized_peptides, optimized_scores), start=1):
                print(f"{i}. Peptide: {peptide}, Score: {score:.3f}")
                write_txt_file("peptides.txt", peptide)

### Создание файлов fasta

In [ ]:
import os

if not os.path.exists("peptides"):
    os.makedirs("peptides")
else:
    for filename in os.listdir("peptides"):
        os.remove(os.path.join("peptides", filename))

with open("peptides.txt", "r") as f:
    peptides = f.read().splitlines()

# Write each peptide to a new file with the .fasta extension
for i, peptide in enumerate(peptides):
    filename = f"peptides/peptide{i+1}.fasta"
    with open(filename, "w") as f:
        f.write(f">peptide_{i+1}\n{peptide}\n")

## Анализ in silico


### Создаем папку для докинга 
Или чистим от прошлых расчетов


In [ ]:
import os
import shutil

# Проверяем, существует ли папка docking
if os.path.exists('docking'):
    # Получаем список всех файлов и папок в папке docking
    files = os.listdir('docking')
    for file in files:
        # Удаляем все файлы и папки, кроме vina.py и params.txt
        if file not in ('vina.py', 'params.txt'):
            path = os.path.join('docking', file)
            if os.path.isdir(path):
                shutil.rmtree(path)
            else:
                os.remove(path)
else:
    # Если папки docking нет, создаем ее
    os.makedirs('docking')

# Создаем папки ligands и receptors внутри папки docking
ligands_dir = os.path.join('docking', 'ligands')
receptors_dir = os.path.join('docking', 'receptors')
os.makedirs(ligands_dir, exist_ok=True)
os.makedirs(receptors_dir, exist_ok=True)

# Копируем файлы vina.py и params.txt в папку docking, если их там нет
for file_name in ('vina.py', 'params.txt'):
    if not os.path.exists(os.path.join('docking', file_name)):
        shutil.copy(file_name, 'docking')


### Преобразуем FASTA в PDB

для этого использовать 
https://colab.research.google.com/github/deepmind/alphafold/blob/main/notebooks/AlphaFold.ipynb#scrollTo=rowN0bVYLe9n

### Запуск AutoDock Vina

In [ ]:
import os

os.chdir('docking')
os.system('python vina.py')